### 1. Установка
Клонируем репозиторий и переходим в нужную директорию.

In [ ]:
# Клонируем репозиторий
# !git clone https://github.com/zhenyacode/acrofinder.git

# Переходим в папку проекта
# %cd acrofinder

### 2. Загрузка большого словаря
Загружаем самый большой словарь словоформ

In [ ]:
# Загрузка для Colab
!wget https://github.com/danakt/russian-words/raw/refs/heads/master/russian.txt -P data/dicts/

"apt-get" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
"apt-get" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
"wget" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [14]:
# Загрузка из под Windows

import urllib.request
import os

# URL файла
url = "https://github.com/danakt/russian-words/raw/refs/heads/master/russian.txt"

# Путь для сохранения
save_dir = "data/dicts"
os.makedirs(save_dir, exist_ok=True)  # Создаём папку, если её нет
save_path = os.path.join(save_dir, "russian.txt")

# Скачиваем файл
print(f"Скачиваем {url}...")
urllib.request.urlretrieve(url, save_path)
print(f"✅ Файл сохранён: {save_path}")

Скачиваем https://github.com/danakt/russian-words/raw/refs/heads/master/russian.txt...
✅ Файл сохранён: data/dicts\russian.txt


In [16]:
# Путь к файлу
file_path = "data/dicts/russian.txt"

# Читаем все слова
with open(file_path, "r", encoding="cp1251") as f:
    words = [line.strip() for line in f if line.strip()]  # Убираем пустые строки

processed_words = set()  # Используем set, чтобы избежать дубликатов

for word in words:
    if not word:  # Пропускаем пустые
        continue

    first_char = word[0]

    # Если слово начинается с ъ или ь — пропускаем (удаляем)
    if first_char in "ъь":
        continue

    # Добавляем оригинал
    processed_words.add(word)

    # Если начинается с ы — добавляем версию с и
    if first_char == "ы" and len(word) > 1:
        new_word = "и" + word[1:]
        processed_words.add(new_word)

    # Если начинается с э — добавляем версию с е
    if first_char == "э" and len(word) > 1:
        new_word = "е" + word[1:]
        processed_words.add(new_word)

# Преобразуем обратно в список и сортируем (опционально)
processed_words = sorted(processed_words)

# Перезаписываем файл
with open(file_path, "w", encoding="utf-8") as f:
    for word in processed_words:
        f.write(word + "\n")

print(f"✅ Обработано {len(processed_words)} слов. Файл перезаписан: {file_path}")

✅ Обработано 1546585 слов. Файл перезаписан: data/dicts/russian.txt


### 3. Загрузка корпуса для исследований

Возьмём все достаточно большие тексты с http://az.lib.ru

Получаем список авторов, их произведений и ссылок на страницы с текстами

In [17]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import time
from urllib.parse import urljoin
from tqdm.notebook import tqdm

# Конфигурация
MAIN_PAGE_URL = "http://az.lib.ru/"
OUTPUT_CSV = "libru_works.csv"
SESSION_DELAY = 0.01  # секунд между запросами

# Настройка сессии
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36'
})

# Шаг 1: Парсим главную страницу и собираем авторов
print("🔍 Шаг 1: Сбор списка авторов с главной страницы...")

try:
    response = session.get(MAIN_PAGE_URL)
    response.encoding = 'cp1251'  # КРИТИЧНО для этого сайта
    soup = BeautifulSoup(response.text, 'html.parser')
except Exception as e:
    print(f"❌ Ошибка загрузки главной страницы: {e}")
    exit(1)

# Собираем авторов: ссылки вида /a/abaza_k_k/, /b/bobylew_n_k/ и т.д.
authors = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    if re.match(r'^/[a-z]/[^/]+/$', href):  # шаблон: /буква/имя_автора/
        author_id = href.strip('/').split('/')[-1]
        full_url = urljoin(MAIN_PAGE_URL, href)
        authors.append((author_id, full_url))

# Убираем дубликаты и сортируем
authors = list(set(authors))
authors.sort(key=lambda x: x[0])

print(f"✅ Найдено {len(authors)} авторов.")


# Шаг 2: Для каждого автора парсим его страницу и извлекаем произведения
print("📚 Шаг 2: Парсинг страниц авторов и сбор произведений...")

works = []

authors_to_parse = list(enumerate(authors, 1))


for idx, (author_id, author_url) in tqdm(authors_to_parse):
    # print(f"→ Обработка {idx}/{len(authors)}: {author_id} ({author_url})")

    try:
        response = session.get(author_url, timeout=10)
        response.encoding = 'cp1251'
        soup = BeautifulSoup(response.text, 'html.parser')

        # Ищем все ссылки на .shtml и .txt (исключаем служебные: about, stat, index и т.п.)
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            if (href.endswith(('.shtml', '.txt')) and
                not href.startswith('/') and  # ← КЛЮЧЕВОЕ: оставляем только относительные ссылки!
                not any(href.startswith(x) for x in ['index', 'about', 'stat', 'linklist', 'cgi-bin'])):

                # Извлекаем имя текста — всё, что до расширения
                text_name = href.rsplit('.', 1)[0]
                full_text_url = urljoin(author_url, href)

                works.append({
                    'author_id': author_id,
                    'text_name': text_name,
                    'full_text_url': full_text_url
                })

        # Вежливая пауза
        time.sleep(SESSION_DELAY)

    except Exception as e:
        print(f"⚠️ Ошибка при обработке {author_url}: {e}")
        continue

# Шаг 3: Сохраняем всё в CSV
print(f"💾 Сохранение {len(works)} произведений в {OUTPUT_CSV}...")

with open(OUTPUT_CSV, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['author_id', 'text_name', 'full_text_url', 'processed']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for work in works:
        work['processed'] = ''
        writer.writerow(work)

print(f"Данные сохранены в {OUTPUT_CSV}")

🔍 Шаг 1: Сбор списка авторов с главной страницы...
✅ Найдено 3890 авторов.
📚 Шаг 2: Парсинг страниц авторов и сбор произведений...


  0%|          | 0/3890 [00:00<?, ?it/s]

KeyboardInterrupt: 

Проходим по списку, обрабатываем каждое произведение -- ищем акростихи с заданными параметрами

In [18]:
INPUT_CSV = "/libru_works.csv"
SESSION_DELAY = 0.01
MIN_SYMBOLS = 5000

# Функция первичной обработки текста
def preprocess_text(text):
    """
    Первичная очистка видимого текста:
    - Убираем лишние переносы и пробелы
    - Нормализуем пробелы
    """
    text = text.replace("\r\n", " ")
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

# Функция безопасного создания имени файла
def sanitize_filename(filename):
    return re.sub(r'[^\w\-_\.]', '_', filename)


# ОСНОВНАЯ ФУНКЦИЯ: принимает URL → возвращает очищенный текст (str) или None
def fetch_and_process_text(url):
    """
    Загружает страницу по URL, извлекает и очищает видимый текст.
    Возвращает строку с текстом или None в случае ошибки/слишком короткого текста.
    """

    try:
        response = session.get(url, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        visible_text = soup.get_text(separator=" ")
        cleaned_text = preprocess_text(visible_text)

        if len(cleaned_text) < MIN_SYMBOLS:
            return None  # Слишком короткий — возвращаем None

        return cleaned_text

    except Exception as e:
        # Можно залогировать ошибку, если нужно, но по условию — просто возвращаем None
        return None



In [ ]:
import pandas as pd 

x = pd.read_csv('libru_works.csv')

x.processed = 0

# x.to_csv('libru_works.csv')

x

,Unnamed: 0.1,Unnamed: 0,author_id,text_name,full_text_url,processed
0,0,0.0,abaza_k_k,text_1890_kazaki-oldorfo,http://az.lib.ru/a/abaza_k_k/text_1890_kazaki-...,1.0
1,1,1.0,abaza_k_k,text_1905_beliy_general_skobelev,http://az.lib.ru/a/abaza_k_k/text_1905_beliy_g...,1.0
2,2,2.0,abaza_k_k,text_1989_abaza_bio,http://az.lib.ru/a/abaza_k_k/text_1989_abaza_b...,1.0
3,3,3.0,abeljar_p,text_1902_historia_calamitatum-oldorfo,http://az.lib.ru/a/abeljar_p/text_1902_histori...,1.0
4,4,4.0,abeljar_p,text_1959_dialogus_inter_philosophum_judaeum_e...,http://az.lib.ru/a/abeljar_p/text_1959_dialogu...,1.0
...,...,...,...,...,...,...
16037,16037,16037.0,doroshewich_w_m,text_1895_8_prizovaya_goryachka,http://az.lib.ru/d/doroshewich_w_m/text_1895_8...,NaN
16038,16038,16038.0,doroshewich_w_m,text_1895_9_vint,http://az.lib.ru/d/doroshewich_w_m/text_1895_9...,NaN
16039,16039,16039.0,doroshewich_w_m,text_1895_9_vv_gostinoy_famusova,http://az.lib.ru/d/doroshewich_w_m/text_1895_9...,NaN
16040,16040,16040.0,doroshewich_w_m,text_1895_cherz_100_let,http://az.lib.ru/d/doroshewich_w_m/text_1895_c...,NaN


In [36]:
import sys
from pathlib import Path

# Добавляем папку src в sys.path
# src_path = Path().resolve().parent / "src"

# sys.path.insert(0, str(src_path))

from acrofinder.scanner import Scanner

INPUT_CSV = "libru_works.csv"

DICTIONARY = 'russian.txt'
LEVELS = ['sentence', 'paragraph', 'word']
min_size = 6
min_n_len = 5
FILTER = True
s = Scanner(min_word_size=min_size, 
            dictionary_name=DICTIONARY)

# s_no_filter = Scanner(min_word_size=7, 
#             dictionary_name=DICTIONARY)

print(f"📂 Чтение данных из {INPUT_CSV}...")
works = []

try:
    with open(INPUT_CSV, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        fieldnames = reader.fieldnames  # Сохраняем порядок и имена колонок
        if 'processed' not in fieldnames:
            fieldnames = list(fieldnames) + ['processed']  # Добавляем колонку, если её нет
        for row in reader:
            # Если колонки processed нет — добавляем по умолчанию "0"
            if 'processed' not in row:
                row['processed'] = '0'
            works.append(row)
    print(f"✅ Загружено {len(works)} записей.")
except Exception as e:
    print(f"❌ Ошибка чтения CSV: {e}")
    exit(1)

# Подсчитаем, сколько уже обработано
initial_processed = sum(1 for w in works if w.get('processed') == '1')
print(f"ℹ️ Уже обработано: {initial_processed}. Начинаем с {initial_processed + 1}-й записи.")

print(f"📥 Начинаем обработку текстов...")

# Путь к файлу результатов
RESULTS_CSV = "results.csv"

# Создадим копию списка для безопасного обновления
updated_works = works.copy()

for idx, work in enumerate(tqdm(works, desc="Обработка текстов"), 1):
    # Пропускаем, если уже обработано
    if work.get('processed') == '1':
        continue

    author_id = work['author_id']
    text_name = work['text_name']
    url = work['full_text_url']

    # ✅ Загружаем и обрабатываем текст
    cleaned_text = fetch_and_process_text(url)

    if cleaned_text is None:
        # tqdm.write(f"⚠️ [{idx}] Пропущен: {author_id} / {text_name}")
        work['processed'] = '0'
    else:
        # ✅ Получили метаданные
        metadata = {
            'author': author_id,
            'title': text_name
        }

        # 🔍 ================ НАЧАЛО БЛОКА ИССЛЕДОВАНИЯ ================
        # Выполняем поиск акростихов с помощью сканера

        try:
            df_results = s.scan_text(
                cleaned_text,
                levels=LEVELS,
                filter_by_neighbours=FILTER,
                min_neighbour_len=min_n_len
            )

            # Если найдены кандидаты — добавляем метаданные и сохраняем в results.csv
            if not df_results.empty:
                # Добавляем колонки с метаданными
                df_results['author_id'] = author_id
                df_results['text_name'] = text_name
                df_results['url'] = url

                # Проверяем, существует ли файл результатов
                file_exists = Path(RESULTS_CSV).exists()

                # Дописываем результаты в CSV
                df_results.to_csv(
                    RESULTS_CSV,
                    mode='a',
                    header=not file_exists,
                    index=False,
                    encoding='utf-8'
                )

                tqdm.write(f"🔍 [{idx}] Найдено {len(df_results)} кандидатов для {text_name}")

            else:
                # tqdm.write(f"🔍 [{idx}] Акростихи не найдены в {text_name}")
                pass

            # ✅ Помечаем как успешно обработанное
            work['processed'] = '1'

        except Exception as scan_error:
            tqdm.write(f"❌ Ошибка сканирования для {text_name}: {scan_error}")
            work['processed'] = '0'  
            continue
        # ================ КОНЕЦ БЛОКА ИССЛЕДОВАНИЯ ================

    # Сразу сохраняем обновлённый CSV
    try:
        with open(INPUT_CSV, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for w in works:  # сохраняем исходный список, который мы мутируем
                writer.writerow(w)
    except Exception as e:
        tqdm.write(f"❌ Ошибка сохранения CSV: {e}")

    # Вежливая пауза
    time.sleep(SESSION_DELAY)

print(f"\nОбработка завершена!")
total_processed = sum(1 for w in works if w.get('processed') == '1')
print(f"   Всего обработано: {total_processed} / {len(works)}")
print(f"   Результаты сохранены в: {INPUT_CSV}")
print(f"   Найденные акростихи сохранены в: {RESULTS_CSV}")

📂 Чтение данных из libru_works.csv...
✅ Загружено 2060 записей.
ℹ️ Уже обработано: 0. Начинаем с 1-й записи.
📥 Начинаем обработку текстов...


Обработка текстов:   0%|          | 0/2060 [00:00<?, ?it/s]

🔍 [4] Найдено 1 кандидатов для text_1902_historia_calamitatum-oldorfo
🔍 [11] Найдено 1 кандидатов для text_1915_ten_veka_sego_oldorfo
🔍 [43] Найдено 1 кандидатов для text_1884_03_vnutr_obozrenie_oldorfo
🔍 [89] Найдено 1 кандидатов для text_0040
🔍 [128] Найдено 1 кандидатов для text_1900_poe_oldorfo
🔍 [203] Найдено 2 кандидатов для text_2008_letopis_zhizni_i_tvorchestva_anny_akhmatovoy
🔍 [267] Найдено 1 кандидатов для text_0105
🔍 [368] Найдено 1 кандидатов для text_1923_sobre_a_mortalidade


KeyboardInterrupt: 